# Overview {#sec-openpose}


In this script, we take the videos prepared in the previous script (@sec-vidprep) and perform OpenPose 2D pose estimation (@ADDREF) on them. 

We are using the BODY_135 model with 135 keypoints.

Demo of this pipeline has been published on [EnvisionBOX](https://www.envisionbox.org/embedded_openpose_to_pose2sim_tracking.html)

See [OpenPose documentation](https://github.com/CMU-Perceptual-Computing-Lab/openpose) for more information.

In [ ]:
#| code-fold: true
#| code-summary: Code to prepare the environment


import os
import subprocess
import glob
import tempfile
from IPython.display import Video
import random

curfolder = os.getcwd()

# Openpose demo.exe location
openposefol = curfolder+'/openpose/'
openpose_demo_loc = openposefol + '/bin/OpenPoseDemo.exe'

# This is the model to employ
model_to_employ = 'BODY_135'

# List folders in a main folder
folderstotrack = glob.glob(curfolder +'/projectdata/*')

# Get all folders per participant, per session
pcnfolders = []

for i in folderstotrack:
    pcn1folders = glob.glob(i + '/P0/*')
    pcn2folders = glob.glob(i + '/P1/*')
    pcnfolders_in_session = pcn1folders + pcn2folders

    # Append to the list
    pcnfolders = pcnfolders + pcnfolders_in_session

# There might be some other things we don't want now
pcnfolders = [x for x in pcnfolders if 'Config' not in x]
pcnfolders = [x for x in pcnfolders if 'opensim' not in x]
pcnfolders = [x for x in pcnfolders if 'xml' not in x]
pcnfolders = [x for x in pcnfolders if 'ResultsInverseDynamics' not in x]
pcnfolders = [x for x in pcnfolders if 'ResultsInverseKinematics' not in x]
pcnfolders = [x for x in pcnfolders if 'sto' not in x]
pcnfolders = [x for x in pcnfolders if 'txt' not in x]

print(pcnfolders[0:10])

In [ ]:
#| code-fold: true
#| code-summary: Code to run OpenPose as a subprocess

def runcommand(command):
    # Run the command using subprocess for OPENPOSE TRACKING
    try:
        subprocess.run(command, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Command execution failed with error code {e.returncode}")
    except FileNotFoundError:
        print("The OpenPoseDemo.exe executable was not found.")


In [ ]:
#| eval: false

for i in pcnfolders:
    os.chdir(openposefol)
    print('working on ' + i)

    # Identify all avi files in folder
    direc = glob.glob(i + '/raw-2d/' +'*.avi')

    # 3 cameras
    video0 = direc[0]
    video1 = direc[1]
    video2 = direc[2]

    videolist = [video0, video1, video2]
    
    # Make a new directory if it doesn't exist
    if not os.path.exists(i+'/pose/'):
        os.makedirs(i+'/pose/')
    if not os.path.exists(i+'/pose/pose_cam1_json/'):
        os.makedirs(i+'/pose/pose_cam1_json/')
    if not os.path.exists(i+'/pose/pose_cam2_json/'):
        os.makedirs(i+'/pose/pose_cam2_json/')
    if not os.path.exists(i+'/pose/pose_cam3_json/'):
        os.makedirs(i+'/pose/pose_cam3_json/')
    

    # Also make directory for openpose videos (pose-2d-trackingvideos)
    if not os.path.exists(i+'/pose-2d-trackingvideos/'):
        os.makedirs(i+'/pose-2d-trackingvideos/')

    # Initialize the pose2 folder
    outputfol1 = i+'/pose/pose_cam1_json/'
    outputfol2 = i+'/pose/pose_cam2_json/'
    outputfol3 = i+'/pose/pose_cam3_json/'

    outputfollist = [outputfol1, outputfol2, outputfol3]

    for it, j in enumerate(outputfollist):
        # Prepare the command
        openposelocation = ' ' + openpose_demo_loc + ' '
        model = '--model_pose' + ' ' + model_to_employ + ' '
        video = '--video ' + videolist[it] + ' '
        todo = '--write_json '
        outputfol = j + ' '
        videoadd = '--write_video '
        videopath = i+'/pose-2d-trackingvideos/' + 'video'+str(it)+'.avi' + ' '
        # Send the command via subprocess
        command = r' '+openposelocation+model+video+todo+outputfol+videoadd+videopath
        print('were going to send this to command prompt: ' + command)
        runcommand(command)
    

The results are saved in  /pose folder in json format, and are later used for triangulation via Pose2sim (@ADD).

The videos with the estimated poses are saved in the /pose-2d-trackingvideos folder

Here is an example of the output:

In [3]:
#| echo: false

# Create a temporary folder
temp_dir = tempfile.mkdtemp()

openposevideos = glob.glob(curfolder + '\\projectdata\\*\\*\\*\\pose-2d-trackingvideos\\*.avi', recursive=True)

# Define input and output paths
sample = random.choice(openposevideos)
mp4_path = os.path.join(temp_dir, "converted_audiovideo.mp4")  # Save MP4 in temp folder

# Convert AVI to MP4 with video and audio
if not os.path.exists(mp4_path):
    subprocess.run([
        "ffmpeg", "-i", sample, 
        "-vcodec", "libx264", "-acodec", "aac", "-b:a", "192k",  # Ensure audio is included
        "-crf", "23", "-preset", "fast", mp4_path
    ])

# Display the video inside Jupyter Notebook
Video(mp4_path, embed=True, width=400)